In [59]:
import nltk  
from nltk.corpus import treebank
from nltk.tag.stanford import StanfordNERTagger
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import json
import operator


In [60]:
import os
import sys
import csv
import random
import itertools
from operator import itemgetter
from collections import defaultdict
import numpy as np
#import scipy
#import scipy.spatial.distance
from numpy.linalg import svd
import matplotlib.pyplot as plt
#from sklearn.decomposition import PCA
#from sklearn.manifold import TSNE
import math


In [61]:
#turn the file into json data
def jFile(path, fileName, data):
    filePathName = path + '/' + fileName +'.json'
    with open (filePathName) as f:
        for line in f:
            data.append(json.loads(line))
    return data

#Modified Hiroshi's development to universal tag. Sentence Tagger
def tagger(sentence):
    sentence = sentence.replace('\n',' ')
    sentence = sentence.replace('\/',' ')
    tokens = nltk.word_tokenize(sentence)
    tagged = nltk.pos_tag(tokens, tagset = 'universal')
    return tagged

#Name Entity Recognition
def NERtagger (sentence):
    nerTagger = StanfordNERTagger('/Users/JQ/Documents/NYU 2018 Spring/NLU/Project Yelp/stanford-ner-2018-02-27/classifiers/english.conll.4class.distsim.crf.ser.gz',
                                  '/Users/JQ/Documents/NYU 2018 Spring/NLU/Project Yelp/stanford-ner-2018-02-27/stanford-ner.jar')
    data = nerTagger.tag(nltk.word_tokenize(sentence))
    return data



#narrow down sentence into noun entities
def nounEntityNarrow (sentence):
    posList = tagger(sentence)
    nerList = NERtagger(sentence)
    potentialFoodList = []

    for i in range(len(posList)):
        if nerList[i][1] == 'O':
            if posList[i][1]  == 'NOUN':
                potentialFoodList.append(posList[i])

    return potentialFoodList

def sentenceBuilder(jsonData):
    sentenceStorage = [] 
    for data in jsonData:
        review = data['text']
        review = review.replace(';','.')
        sentences = nltk.sent_tokenize(review)
        for eachS in sentences:
            sentenceStorage.append(eachS)
        
    return sentenceStorage


In [62]:
def euclidean(u, v):    
    a=len(u)
    b=len(v)
    if a != b:
        print("input vectors have different dimensions")
    s=0
    for i in range(a):
        s+=np.absolute(u[i]-v[i])**2
    s=np.sqrt(s)
    
    return s

def vector_length(u):
    a=len(u)
    s=0
    for i in range(a):
        s+=u[i]**2
    s=np.sqrt(s)
    return s

def length_norm(u):
    """L2 norm of the 1d np.array `u`. Returns a float."""
    return u / vector_length(u)
 
    
def cosine(u, v):        
    
    """Cosine distance between 1d np.arrays `u` and `v`, which must have 
    the same dimensionality. Returns a float."""
    return euclidean(length_norm(u),length_norm(v))

def neighbors(word, mat, rownames, distfunc=cosine):    
    """Tool for finding the nearest neighbors of `word` in `mat` according 
    to `distfunc`. The comparisons are between row vectors.
    
    Parameters
    ----------
    word : str
        The anchor word. Assumed to be in `rownames`.
        
    mat : np.array
        The vector-space model.
        
    rownames : list of str
        The rownames of mat.
            
    distfunc : function mapping vector pairs to floats (default: `cosine`)
        The measure of distance between vectors. Can also be `euclidean`, 
        `matching`, `jaccard`, as well as any other distance measure  
        between 1d vectors.
        
    Raises
    ------
    ValueError
        If word is not in rownames.
    
    Returns
    -------    
    list of tuples
        The list is ordered by closeness to `word`. Each member is a pair 
        (word, distance) where word is a str and distance is a float.
    
    """
    if word not in rownames:
        raise ValueError('%s is not in this VSM' % word)
    w = mat[rownames.index(word)]
    dists = [(rownames[i], distfunc(w, mat[i])) for i in range(len(mat))]
    y = [x for x in dists if str(x[1]) != 'nan']
            
    return sorted(y, key=itemgetter(1), reverse=False)

In [75]:
def prob_norm(u):
    """Normalize 1d np.array `u` into a probability distribution. Assumes 
    that all the members of `u` are positive. Returns a 1d np.array of 
    the same dimensionality as `u`."""
    return u / np.sum(u)
def pmi(mat, rownames=None, positive=True):
    """Pointwise Mutual Information with Positive on by default.
    
    Parameters
    ----------
    mat : 2d np.array
       The matrix to operate on.
           
    rownames : list of str or None
        Not used; it's an argument only for consistency with other methods 
        defined here.
        
    positive : bool (default: True)
        Implements Positive PMI.
        
    Returns
    -------    
    (np.array, list of str)
       The first member is the PMI-transformed version of `mat`, and the 
       second member is `rownames` (unchanged).
    
    """    
    # Joint probability table:
    msum=np.sum(mat)
    pmat=mat/msum
    
    colSum=0
    rowSum=0
    # Pre-compute column sums:
    for i in range(pmat.shape[0]):
        colSum += pmat[i,:]
    # Vectorize this function so that it can be applied rowwise: 
    for i in range(pmat.shape[1]):
        rowSum += pmat[:,i]
    
    p=np.zeros((pmat.shape[0],pmat.shape[1]))
    for i in range(pmat.shape[0]):
        for j in range(pmat.shape[1]):
            p[i,j]=_pmi_log(pmat[i,j]/(rowSum[i]*colSum[j]),positive=True)
    
    return (p, rownames)

def _pmi_log(x, positive=True):
    """With `positive=False`, return log(x) if possible, else 0.
    With `positive=True`, log(x) is mapped to 0 where negative."""

    if positive == False:
        try:
            return np.log2(x)
        except:
            return 0
    
    if positive == True:
        if x>1:
            return np.log2(x)
        else:
            return 0
            
        


In [80]:
def lsa(mat=None, rownames=None, k=100):
    """Latent Semantic Analysis using pure scipy.
    
    Parameters
    ----------
    mat : 2d np.array
       The matrix to operate on.
           
    rownames : list of str or None
        Not used; it's an argument only for consistency with other methods 
        defined here.
        
    k : int (default: 100)
        Number of dimensions to truncate to.
        
    Returns
    -------    
    (np.array, list of str)
        The first member is the SVD-reduced version of `mat` with 
        dimension (m x k), where m is the rowcount of mat and `k` is 
        either the user-supplied k or the column count of `mat`, whichever 
        is smaller. The second member is `rownames` (unchanged).

    """    
    rowmat, singvals, colmat = svd(mat, full_matrices=False)
    singvals = np.diag(singvals)
    trunc = np.dot(rowmat[:, 0:k], singvals[0:k, 0:k])
    return (trunc, rownames)

In [85]:
path= "/Users/JQ/Downloads/done"

data = []
data = jFile(path, '4JNXUYY8wbaaDmk3BPzlWw', data)
    
sentences = sentenceBuilder(data)
count_model = CountVectorizer(ngram_range=(1,1))
X = count_model.fit_transform(sentences)
Xc = (X.T * X) # this is co-occurrence matrix in sparse csr format
#Xc.setdiag(0) # sometimes you want to fill same word cooccurence to 0


In [86]:
'''
nounList = []
for i in sentences:
    nounList.append(nounEntityNarrow(i))
print(nounList)
'''


'\nnounList = []\nfor i in sentences:\n    nounList.append(nounEntityNarrow(i))\nprint(nounList)\n'

In [97]:
dishName =[]
file = "/Users/JQ/Downloads/results/final1.txt"
with open(file) as f:
    dishName = [line.rstrip() for line in f]
#for n, line in enumerate(dishName, 1):
    
dish_count = CountVectorizer(ngram_range=(1,1))
Y = dish_count.fit_transform(dishName)
Yc = (Y.T * Y)

dishNameList = dish_count.get_feature_names()

In [98]:
len(dishNameList)

566

In [99]:
tempList=[]
tempList2 = []
for n in dishNameList:
    tempList.append(tagger(n))
for n in tempList:
    if n[0][1] == 'NOUN':
        tempList2.append(n[0][0])


In [100]:
dishNameList=tempList2

In [107]:
excludeList = ['and','for','with','without','of','in','good','best', 'great', 'or','the','meat','dish','appetizer','appetite','dessert',
               'chops','dates','fork','pan','fed','chew','diet','restaurant','diner','dinner',
               'brunch','summer','glasses','water','vitamin','sub','straw',
               'ate','eat','gastronomy','fast','provisions','menu','nourishment','nutrition',
               'breakfast','dog','lunch','entree','order','refreshments','hungry','appetizer']


In [108]:
for n in dishNameList:
    if n in excludeList:
        dishNameList.remove(n)


In [109]:
foodMatrix =  np.zeros(shape=(len(dishNameList),len(dishNameList)))

featureIndex=[]
newDishNameList = []
i=0
for n in dishNameList:
    if count_model.vocabulary_.get(n) != None:
        featureIndex.append(count_model.vocabulary_.get(n))
        newDishNameList.append(n)

dishNameList=newDishNameList
    
Ac=Xc.copy()
#print(Zc[696])
#Zc[dish_count.vocabulary_.get('temperatures')]

#for n in featureIndex:
 #   Ac=np.delete(Bc,1,axis=0)
  #  Ac=np.delete(Ac,1,axis=1)
    
Zc=Ac[featureIndex,:][:,featureIndex]


In [110]:
#dish_count.vocabulary_.get('temperatures')
np.shape(Zc)
#len(newDishNameList)
#len(featureIndex)

(367, 367)

In [111]:
Zc.setdiag(0)
B=Zc.toarray()
matrix=[B,dishNameList]

for n in range(380):
    sortedarray=np.flipud(np.argsort(matrix[0][n]))
    print(matrix[1][n]+" | " + matrix[1][sortedarray[0]]+" " +matrix[1][sortedarray[1]]+" " +matrix[1][sortedarray[2]]+" " +matrix[1][sortedarray[3]])

acorn | soup squash apple spices
almond | trout soup cake grape
almonds | trout brown beans butter
anchovy | pate chicken fillet onion
anise | fennel zucchini fava dough
apple | chicken crepe cheese sandwich
apricot | sweet pancake crepe feed
artichoke | lemon soup sauce potato
asparagus | salad steak soup cheese
avocado | salmon salad turkey tomato
bacon | salad egg cheese benedict
bagel | salmon platter cream cheese
bagels | toast zucchini fava dough
bake | cheese steak zucchini fava
banana | crepe waffle steak caramel
barbecue | steak zucchini feast dough
barley | steak zucchini feast dough
basil | salad cheese tomato sauce
batter | cod beer crepe toast
bean | vanilla cream ice salad
beans | steak potatoes trout chicken
beef | hash corn benedict steak
beer | wine food batter steak
beet | salad lettuce butter cheese
bell | pepper scallops celery julienne
benedict | salmon egg beef bacon
berry | waffle pancake hash toast
biscuit | shrimp lemon bread crepe
bitter | coffee sweet scallop

IndexError: index 367 is out of bounds for axis 0 with size 367

In [112]:
neighbors(word='cheesecake', mat=matrix[0], rownames=matrix[1], distfunc=cosine)

/Users/JQ/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


[('cheesecake', 0.0),
 ('basil', 0.85054719033781379),
 ('tomato', 0.8994451032707742),
 ('mustard', 0.92947855938788781),
 ('dip', 0.93163117041736954),
 ('mussels', 0.94253270359595331),
 ('meatball', 0.95464018986022436),
 ('plate', 0.98781114903512623),
 ('pasta', 0.99517197739462848),
 ('spaghetti', 1.0),
 ('tarragon', 1.0),
 ('butter', 1.00860889012407),
 ('avocado', 1.0092454577434316),
 ('quinoa', 1.0094505771271953),
 ('pear', 1.0116589590739602),
 ('greens', 1.0196192713847525),
 ('platter', 1.0202846634051681),
 ('spinach', 1.0276363696260065),
 ('cheddar', 1.0281979179214169),
 ('steak', 1.0283461007906729),
 ('capers', 1.028644896873969),
 ('onions', 1.0351383859350516),
 ('watercress', 1.0351743017013826),
 ('zucchini', 1.0370630033695887),
 ('garlic', 1.0381077576931439),
 ('vinegar', 1.0381300728610878),
 ('food', 1.0391541413884702),
 ('pop', 1.0416374503657078),
 ('citrus', 1.0493173257165875),
 ('pea', 1.052204333652341),
 ('slice', 1.054951582291537),
 ('egg', 1.059

In [113]:
ww_ppmi = pmi(mat=matrix[0], rownames=matrix[1], positive=True)

/Users/JQ/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in double_scalars


In [179]:
np.shape(ww_ppmi[0])

(367, 367)

In [180]:
finalList1=[]
for n in range(367):
    sortedarray=np.flipud(np.argsort(ww_ppmi[0][n]))
    finalList1.append([dishNameList[n], 
                       dishNameList[sortedarray[0]],
                       dishNameList[sortedarray[1]],
                       dishNameList[sortedarray[2]],
                       dishNameList[sortedarray[3]]])
    

In [182]:
finalList1

[['acorn', 'spices', 'squash', 'apple', 'soup'],
 ['almond', 'grape', 'trout', 'cake', 'caramel'],
 ['almonds', 'pumpkin', 'trout', 'sole', 'crunch'],
 ['anchovy', 'fillet', 'pate', 'beef', 'chicken'],
 ['anise', 'fennel', 'zucchini', 'fava', 'dough'],
 ['apple', 'chutney', 'dill', 'cobbler', 'acorn'],
 ['apricot', 'pancake', 'sweet', 'crepe', 'feed'],
 ['artichoke', 'sole', 'blt', 'bitter', 'bean'],
 ['asparagus', 'salami', 'string', 'pasta', 'horseradish'],
 ['avocado', 'sesame', 'relish', 'salsa', 'turkey'],
 ['bacon', 'watermelon', 'sushi', 'curds', 'romaine'],
 ['bagel', 'oats', 'lox', 'oatmeal', 'capers'],
 ['bagels', 'toast', 'zucchini', 'fava', 'dough'],
 ['bake', 'cheese', 'steak', 'zucchini', 'fava'],
 ['banana', 'whiskey', 'batter', 'waffle', 'caramel'],
 ['barbecue', 'steak', 'zucchini', 'feast', 'dough'],
 ['barley', 'steak', 'zucchini', 'feast', 'dough'],
 ['basil', 'zucchini', 'oil', 'ginger', 'parsley'],
 ['batter', 'beer', 'cod', 'banana', 'crisp'],
 ['bean', 'casserol

In [154]:
neighbors(word='banana', mat=ww_ppmi[0], rownames=ww_ppmi[1], distfunc=cosine)

/Users/JQ/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


[('banana', 0.0),
 ('chocolate', 1.0817747917625984),
 ('waffle', 1.0879705638401431),
 ('cinnamon', 1.1038046052248818),
 ('blueberry', 1.1059321373801634),
 ('torte', 1.1589017639509596),
 ('strawberry', 1.1703568885306848),
 ('syrup', 1.1864294008765337),
 ('crepe', 1.1930259008409829),
 ('cream', 1.1944789506755065),
 ('pastry', 1.2079002620802306),
 ('sausage', 1.213472528573299),
 ('mango', 1.2244983919632006),
 ('hazelnut', 1.2247633932657376),
 ('toast', 1.2313224169687074),
 ('buckwheat', 1.2372768604155466),
 ('blood', 1.239407284801864),
 ('berry', 1.2412479248825059),
 ('maple', 1.2441991936890313),
 ('broil', 1.2484262717057732),
 ('vanilla', 1.2488360682843578),
 ('apricot', 1.250245913248905),
 ('caramel', 1.2540385253855126),
 ('bacon', 1.2656012199060396),
 ('jam', 1.2662363214497512),
 ('benedict', 1.2662520206815735),
 ('plate', 1.266618604269887),
 ('sugar', 1.2685980077594186),
 ('cake', 1.2687196284065696),
 ('crisp', 1.271204338804873),
 ('bean', 1.27287580478368

In [119]:
gnmat_lsa = lsa(mat=ww_ppmi[0], rownames=ww_ppmi[1], k=np.shape(ww_ppmi[0])[0])

In [193]:
neighbors(word='salmon', mat=gnmat_lsa[0], rownames=gnmat_lsa[1], distfunc=cosine)

[('salmon', 0.0),
 ('pea', 1.1309284082506996),
 ('tuna', 1.1528380320740592),
 ('lemon', 1.1565869439671277),
 ('citrus', 1.1723074377767881),
 ('spinach', 1.1889321808942666),
 ('avocado', 1.20068006563397),
 ('salad', 1.2017872135627914),
 ('halibut', 1.204768852749091),
 ('lox', 1.2057397548593738),
 ('bacon', 1.227467176125622),
 ('asparagus', 1.2342566652462743),
 ('kale', 1.2431781795475432),
 ('greens', 1.2436305818153297),
 ('peas', 1.2442195354372372),
 ('benedict', 1.2446358524527583),
 ('roast', 1.248767852910782),
 ('tarragon', 1.2528216132079586),
 ('egg', 1.2557704142834629),
 ('cloves', 1.2583478479578067),
 ('capers', 1.2650768179704208),
 ('thyme', 1.2651629771244712),
 ('cauliflower', 1.2683043217344752),
 ('butter', 1.2691378525798984),
 ('potatoes', 1.2712111202481926),
 ('shrimp', 1.2742045105053785),
 ('bagel', 1.2801580270108674),
 ('seaweed', 1.2809517550540088),
 ('platter', 1.2842503896254132),
 ('chick', 1.2846401321125402),
 ('quinoa', 1.2866378674748746),


In [175]:
FinalNames=[]
for n in range(len(dishNameList)):
    a=[]
    a=neighbors(word=gnmat_lsa[1][n], mat=gnmat_lsa[0], rownames=gnmat_lsa[1], distfunc=cosine)
    pname=[]
    for m in a:
        if m[1]<1.2:
            pname.append(m[0])
    if len(pname) > 1:
        if len(pname) > 6:
            FinalNames.append(pname[0:6])
        else:
            FinalNames.append(pname)


In [212]:
def measure(listItems):
    menuDiscriptions =[]
    file = "/Users/JQ/Downloads/4JNXUYY8wbaaDmk3BPzlWw.txt"
    with open(file) as f:
        menuDiscriptions = [line.rstrip().lower() for line in f]
    #check how many items are matching
    mainDishMatch = []
    fullPredictDishMatch = []
    for n in listItems:
        accuracyList=[]
        for m in menuDiscriptions:
            if n[0] in m:
                i=len(n)
                j=0
                for x in n:
                    if x in m:
                        j=j+1
                accuracyList.append([m,j/len(m.split())])
        if len(accuracyList)>0:
            a=sorted(accuracyList,key=itemgetter(1),reverse=True)
            mainDishMatch.append([n,a[0][0],a[0][1]])
                    
    return mainDishMatch

def fullMatch(listItems):
    menuDiscriptions =[]
    file = "/Users/JQ/Downloads/4JNXUYY8wbaaDmk3BPzlWw.txt"
    with open(file) as f:
        menuDiscriptions = [line.rstrip().lower() for line in f]
    #check how many items are matching
    mainDishMatch = []
    for n in listItems:
        accuracyList=[]
        for m in menuDiscriptions:
            i=len(n)
            j=0
            for x in n:
                if x in m:
                    j=j+1
            if j == i:
                accuracyList.append(m)
        if len(accuracyList) > 0:
            mainDishMatch.append([n,accuracyList])
                    
    return mainDishMatch

In [216]:
posName =[]
file = "/Users/JQ/Downloads/results/final2.txt"
with open(file) as f:
    posName = [line.rstrip() for line in f]
posName1=[]
for eachitem in posName:
    posName1.append(eachitem.split())
final=measure(posName1)
sorted(final,key=itemgetter(2),reverse=True)

[[['chocolate', 'mousse'], 'chocolate mousse', 1.0],
 [['chocolate', 'mousse', 'and', 'coffee'], 'chocolate mousse', 1.0],
 [['garlic', 'spinach'], 'garlic spinach', 1.0],
 [['chocolate', 'mousse', 'dessert'], 'chocolate mousse', 1.0],
 [['bacon', 'cheese', 'and', 'egg', 'crepe'],
  'egg, bacon & cheese crepe',
  0.8],
 [['egg', 'cheese', 'and', 'bacon', 'crepe'],
  'egg, bacon & cheese crepe',
  0.8],
 [['benedict', 'and', 'egg', 'benedict'],
  'eggs benedict canadian bacon, hollandaise sauce',
  0.6666666666666666],
 [['chicken', 'and', 'apple', 'crepe'], 'chicken, brie & apple crepe', 0.6],
 [['chicken', 'apple', 'crepe'], 'chicken, brie & apple crepe', 0.6],
 [['cream', 'and', 'vanilla', 'ice', 'cream'],
  'gala apple tarte tatin vanilla ice cream',
  0.5714285714285714],
 [['steak'], 'steak frites', 0.5],
 [['chocolate', 'tart'], 'chocolate mousse', 0.5],
 [['steak', 'sandwich'], 'steak frites', 0.5],
 [['egg', 'and', 'bacon'],
  'eggs benedict canadian bacon, hollandaise sauce',


In [224]:
fullMatch(FinalNames)

[[['beef', 'hash'],
  ['corned beef hash & eggs poached eggs roasted red pepper',
   'corned beef hash & poached eggs* roast red pepper']],
 [['chicken', 'pate', 'liver'],
  ['chicken liver mousse pate burgundy red wine mustard, cornichons']],
 [['onion', 'soup'],
  ['onion soup au gratin baked with gruyere cheese, gluten-free croutons',
   'onion soup au gratin the french classic baked with gruyere cheese',
   'onion soup au gratin the french classic baked with gruyere cheese',
   'french onion soup the french classic']],
 [['pate', 'chicken', 'mustard', 'mousse', 'liver'],
  ['chicken liver mousse pate burgundy red wine mustard, cornichons']],
 [['peas', 'scallops'], ['seafood crepe scallops, shrimp, peas & cream']],
 [['prime', 'sandwich'],
  ['prime steak sandwich* caramelized onions & peppercorn mayo']],
 [['relish', 'avocado'],
  ['egg white omelette avocado & tomato relish, chicken sausage']],
 [['sauce', 'cheese', 'tomato'],
  ['baked goat cheese tomato sauce, warm herb garlic 

In [222]:
final=measure(FinalNames)
sorted(final,key=itemgetter(2),reverse=True)


[[['brown', 'sugar', 'fruit', 'yogurt', 'cantaloupe', 'mint'],
  'fresh fruit salad brown sugar yogurt',
  0.6666666666666666],
 [['fruit', 'sugar', 'brown', 'pineapple', 'yogurt', 'cantaloupe'],
  'fresh fruit salad brown sugar yogurt',
  0.6666666666666666],
 [['sugar', 'brown', 'fruit', 'mint', 'yogurt', 'cantaloupe'],
  'fresh fruit salad brown sugar yogurt',
  0.6666666666666666],
 [['yogurt', 'cantaloupe', 'melon', 'sugar', 'fruit', 'brown'],
  'fresh fruit salad brown sugar yogurt',
  0.6666666666666666],
 [['cream', 'ice', 'chocolate', 'vanilla', 'tart', 'caramel'],
  'gala apple tarte tatin vanilla ice cream',
  0.5714285714285714],
 [['pate', 'chicken', 'mustard', 'mousse', 'liver'],
  'chicken liver mousse pate burgundy red wine mustard, cornichons',
  0.5555555555555556],
 [['chocolate', 'ice', 'cream', 'banana', 'caramel', 'cake'],
  'chocolate waffle whipped cream',
  0.5],
 [['egg', 'bacon', 'muffin', 'benedict', 'mushroom', 'ham'],
  'eggs benedict canadian bacon, holla

In [223]:
i=0
for n in final:
    if n[2]>0.4:
        i=i+1
i/len(final)

0.1564625850340136

In [196]:
menuDiscriptions =[]
file = "/Users/JQ/Downloads/4JNXUYY8wbaaDmk3BPzlWw.txt"
with open(file) as f:
    menuDiscriptions = [line.rstrip().lower() for line in f]
menuDiscriptions


['fruit coupe vanilla syrup',
 '1 egg scrambled hash browns & bacon',
 'creme brulee french toast blueberries & whipped cream',
 'chocolate waffle whipped cream',
 'banana foster waffle whipped cream',
 'cheeseburger & frites',
 'grilled cheese sandwich & frites fresh green beans and apple slices',
 'mac ‘n cheese fresh green beans and apple slices',
 'pasta with butter or tomato sauce fresh green beans and apple slices',
 'cheeseburger & frites fresh green beans and apple slices',
 'chicken fingers & frites fresh green beans and apple slices',
 'steak & frites fresh green beans and apple slices',
 'hot fudge sundae',
 'chocolate mousse',
 'profiterole',
 'onion soup au gratin baked with gruyere cheese, gluten-free croutons',
 'salad maison arugula, savory eggplant jam, pepper drops, sherry vinaigrette',
 'steamed artichoke shaved vegetables, lemon-dijon vinaigrette',
 'oysters du jour shallot-black pepper mignonette, fresh horseradish',
 'shrimp cocktail lemon dijonnaise, cocktail sau

In [55]:
np.shape(matrix[0])

(1288, 1288)

In [54]:
len(dishNameList)

1288